# Banking Feature Store Demo

This notebook demonstrates the banking feature store with three different teams and their use cases:

## Teams:
- **Customer Experience Team**: Customer Charter Model, Customer Behavior Prediction
- **Risk & Fraud Team**: Transaction Type Prediction, Fraud Detection  
- **Operations Team**: Call Prediction, Branch Optimization

## Use Cases:
1. **Customer Charter Model**: Predict customer satisfaction and loyalty
2. **Customer Behavior Prediction**: Predict spending patterns and financial needs
3. **Call Prediction Model**: Predict likelihood of customer service calls
4. **Transaction Type Prediction**: Classify transactions and detect fraud

## What This Demo Shows:
- **Raw Data Exploration**: Examine actual data from Parquet files
- **Feature Store Operations**: Apply configurations and materialize features
- **Offline Feature Retrieval**: Get historical features for model training
- **Online Feature Retrieval**: Get real-time features for model inference
- **Feature Services**: Use pre-defined feature sets for different ML models


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from feast import FeatureStore
import os

# Initialize Feast feature store
fs = FeatureStore(repo_path="feature_repo")

print("✅ Feature Store initialized successfully!")


✅ Feature Store initialized successfully!


## 1. Explore Raw Data from Parquet Files

Let's first examine the actual data in our Parquet files to understand what we're working with.


In [2]:
# Check if data files exist
data_dir = "feature_repo/data"
parquet_files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]
print(f"📁 Found {len(parquet_files)} Parquet files:")
for file in parquet_files:
    print(f"  - {file}")

print("\n" + "="*60)
print("📊 EXAMINING RAW DATA FROM PARQUET FILES")
print("="*60)

# Load and examine each dataset
datasets = {}

for file in parquet_files:
    file_path = os.path.join(data_dir, file)
    df = pd.read_parquet(file_path)
    datasets[file.replace('.parquet', '')] = df
    
    print(f"\n🔍 {file.upper()}")
    print(f"   Shape: {df.shape}")
    print(f"   Columns: {list(df.columns)}")
    
    # Handle different timestamp column names
    timestamp_col = None
    if 'event_timestamp' in df.columns:
        timestamp_col = 'event_timestamp'
    elif 'created_timestamp' in df.columns:
        timestamp_col = 'created_timestamp'
    
    if timestamp_col:
        print(f"   Date range: {df[timestamp_col].min()} to {df[timestamp_col].max()}")
    else:
        print(f"   ⚠️  No timestamp column found")
    
    print(f"   Sample data:")
    print(df.head(3).to_string(index=False))
    print("-" * 40)


📁 Found 5 Parquet files:
  - call_center.parquet
  - transactions.parquet
  - atm_usage.parquet
  - branch_visits.parquet
  - customers.parquet

📊 EXAMINING RAW DATA FROM PARQUET FILES

🔍 CALL_CENTER.PARQUET
   Shape: (12000, 9)
   Columns: ['customer_id', 'call_id', 'call_type', 'call_duration_minutes', 'resolution_time_hours', 'is_resolved', 'customer_satisfaction_score', 'escalation_level', 'event_timestamp']
   Date range: 2025-06-18 19:23:53.596551 to 2025-09-16 19:23:53.596551
   Sample data:
customer_id       call_id call_type  call_duration_minutes  resolution_time_hours  is_resolved  customer_satisfaction_score  escalation_level               event_timestamp
CUST_000752 CALL_00000001   INQUIRY                   20.1                   20.6            1                          1.1                 1 2025-06-18 19:23:53.596551000
CUST_000284 CALL_00000002   INQUIRY                    7.9                   52.3            1                          2.8                 3 2025-06-18

## 2. Apply Feature Store Configuration

Now let's apply the feature store configuration to register all entities, feature views, and feature services.


In [3]:
# List registered components
print(f"\n📋 REGISTERED COMPONENTS:")
print(f"   Entities: {len(fs.list_entities())}")
print(f"   Feature Views: {len(fs.list_feature_views())}")
print(f"   Feature Services: {len(fs.list_feature_services())}")

print(f"\n🏷️ ENTITIES:")
for entity in fs.list_entities():
    print(f"   - {entity.name}")

print(f"\n👁️ FEATURE VIEWS:")
for fv in fs.list_feature_views():
    print(f"   - {fv.name}")

print(f"\n🔧 FEATURE SERVICES:")
for fs_service in fs.list_feature_services():
    print(f"   - {fs_service.name}")



📋 REGISTERED COMPONENTS:
   Entities: 4
   Feature Views: 17
   Feature Services: 8

🏷️ ENTITIES:
   - transaction
   - atm_location
   - branch
   - customer

👁️ FEATURE VIEWS:
   - customer_behavioral_profile
   - atm_time_patterns
   - customer_demographics_fv
   - call_center_90d
   - customer_atm_interaction
   - branch_performance
   - transaction_details
   - atm_location_performance
   - transaction_7d_aggregations
   - customer_transaction_interaction
   - call_center_predictive
   - transaction_90d_patterns
   - branch_service_preferences
   - atm_usage_30d
   - transaction_30d_aggregations
   - customer_branch_interaction
   - branch_visits_90d

🔧 FEATURE SERVICES:
   - customer_behavior_service
   - atm_optimization_service
   - call_prediction_service
   - customer_charter_service
   - branch_optimization_service
   - comprehensive_banking_service
   - transaction_prediction_service
   - risk_compliance_service


## 3. Materialize Features to Online Store

Let's materialize features to the online store so we can retrieve them for real-time inference.


In [4]:
# Materialize features to online store
print("🔄 Materializing features to online store...")
end_date = datetime.now()
fs.materialize_incremental(end_date=end_date)
print("✅ Features materialized successfully!")


🔄 Materializing features to online store...
Materializing 17 feature views to 2025-09-16 21:41:40+00:00 into the sqlite online store.

customer_behavioral_profile from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
atm_time_patterns from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
customer_demographics_fv from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
call_center_90d from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
customer_atm_interaction from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
branch_performance from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
transaction_details from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
atm_location_performance from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
transaction_7d_aggregations from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
customer_transaction_interaction from 2025-09-16 21:33:54+00:00 to 2025-09-16 21:41:40+00:00:
call_center_predic

## 4. Offline Feature Retrieval (Historical Data for Training)

Let's retrieve historical features from the offline store for model training. This demonstrates how to get features for a specific time range.


In [5]:
# Get some sample customer IDs and transaction IDs from our data
sample_customers = datasets['customers']['customer_id'].head(5).tolist()
sample_transactions = datasets['transactions']['transaction_id'].head(5).tolist()

print(f"🎯 Sample entities for feature retrieval:")
print(f"   Customer IDs: {sample_customers}")
print(f"   Transaction IDs: {sample_transactions}")

# Use timestamps that are within TTL windows for all feature views
# All data has been materialized to 2025-09-16, so use recent timestamps
recent_timestamp = pd.Timestamp('2025-09-15')  # Within TTL windows for all features

print(f"\n📅 Using recent timestamp within TTL windows:")
print(f"   Timestamp: {recent_timestamp}")
print(f"   This ensures data is available for all feature views")

# Create entity dataframe for offline retrieval with recent timestamp
entity_df = pd.DataFrame({
    'customer_id': sample_customers,
    'event_timestamp': [recent_timestamp] * len(sample_customers)
})

print(f"\n📋 Entity DataFrame for offline retrieval:")
print(entity_df)


🎯 Sample entities for feature retrieval:
   Customer IDs: ['CUST_000001', 'CUST_000002', 'CUST_000003', 'CUST_000004', 'CUST_000005']
   Transaction IDs: ['TXN_00000001', 'TXN_00000002', 'TXN_00000003', 'TXN_00000004', 'TXN_00000005']

📅 Using recent timestamp within TTL windows:
   Timestamp: 2025-09-15 00:00:00
   This ensures data is available for all feature views

📋 Entity DataFrame for offline retrieval:
   customer_id event_timestamp
0  CUST_000001      2025-09-15
1  CUST_000002      2025-09-15
2  CUST_000003      2025-09-15
3  CUST_000004      2025-09-15
4  CUST_000005      2025-09-15


In [6]:
# Retrieve features from offline store for training
print("🔄 Retrieving features from offline store...")

# Get features for customer behavioral profile (for customer charter model)
try:
    training_df = fs.get_historical_features(
        entity_df=entity_df,
        features=[
            "customer_behavioral_profile:age",
            "customer_behavioral_profile:income", 
            "customer_behavioral_profile:credit_score",
            "customer_behavioral_profile:account_tenure_days",
            "customer_behavioral_profile:risk_profile",
            "customer_behavioral_profile:customer_segment"
        ]
    ).to_df()
    
    print("✅ Successfully retrieved customer behavioral profile features!")
    print(f"📊 Training DataFrame shape: {training_df.shape}")
    print(f"📋 Columns: {list(training_df.columns)}")
    if len(training_df) > 0:
        print(f"\n📈 Sample training data:")
        print(training_df.head())
    else:
        print("⚠️  No data returned - checking TTL settings...")
        fv = fs.get_feature_view("customer_behavioral_profile")
        print(f"   TTL: {fv.ttl}")
        print(f"   Time difference: {datetime.now() - recent_timestamp}")
    
except Exception as e:
    print(f"❌ Error retrieving features: {e}")

# Also try getting features from customer_demographics_fv (which has fewer features)
print(f"\n🔄 Trying customer_demographics_fv features...")
try:
    demo_df = fs.get_historical_features(
        entity_df=entity_df,
        features=[
            "customer_demographics_fv:age",
            "customer_demographics_fv:income", 
            "customer_demographics_fv:credit_score"
        ]
    ).to_df()
    
    print("✅ Successfully retrieved customer demographic features!")
    print(f"📊 Demo DataFrame shape: {demo_df.shape}")
    print(f"📋 Columns: {list(demo_df.columns)}")
    if len(demo_df) > 0:
        print(f"\n📈 Sample demographic data:")
        print(demo_df.head())
    else:
        print("⚠️  No data returned - checking TTL settings...")
        fv = fs.get_feature_view("customer_demographics_fv")
        print(f"   TTL: {fv.ttl}")
        print(f"   Time difference: {datetime.now() - recent_timestamp}")
    
except Exception as e:
    print(f"❌ Error retrieving demographic features: {e}")

# Try getting transaction features with the same recent timestamp
print(f"\n🔄 Trying transaction features with recent timestamp...")
try:
    transaction_df = fs.get_historical_features(
        entity_df=entity_df,  # Use the same entity_df with recent_timestamp
        features=[
            "transaction_7d_aggregations:amount",
            "transaction_7d_aggregations:transaction_type",
            "transaction_7d_aggregations:merchant_category",
            "transaction_7d_aggregations:is_fraud",
            "transaction_7d_aggregations:location"
        ]
    ).to_df()
    
    print("✅ Successfully retrieved transaction features!")
    print(f"📊 Transaction DataFrame shape: {transaction_df.shape}")
    print(f"📋 Columns: {list(transaction_df.columns)}")
    if len(transaction_df) > 0:
        print(f"\n📈 Sample transaction data:")
        print(transaction_df.head())
    else:
        print("⚠️  No transaction data returned - checking TTL settings...")
        fv = fs.get_feature_view("transaction_7d_aggregations")
        print(f"   TTL: {fv.ttl}")
        print(f"   Time difference: {datetime.now() - recent_timestamp}")
    
except Exception as e:
    print(f"❌ Error retrieving transaction features: {e}")

print("\n" + "="*60)
print("📊 OFFLINE FEATURE RETRIEVAL COMPLETE")
print("="*60)


🔄 Retrieving features from offline store...
✅ Successfully retrieved customer behavioral profile features!
📊 Training DataFrame shape: (0, 8)
📋 Columns: ['customer_id', 'event_timestamp', 'age', 'income', 'credit_score', 'account_tenure_days', 'risk_profile', 'customer_segment']
⚠️  No data returned - checking TTL settings...
   TTL: 180 days, 0:00:00
   Time difference: 1 days 21:41:54.208832

🔄 Trying customer_demographics_fv features...
✅ Successfully retrieved customer demographic features!
📊 Demo DataFrame shape: (0, 5)
📋 Columns: ['customer_id', 'event_timestamp', 'age', 'income', 'credit_score']
⚠️  No data returned - checking TTL settings...
   TTL: 365 days, 0:00:00
   Time difference: 1 days 21:41:54.236965

🔄 Trying transaction features with recent timestamp...
✅ Successfully retrieved transaction features!
📊 Transaction DataFrame shape: (5, 7)
📋 Columns: ['customer_id', 'event_timestamp', 'amount', 'transaction_type', 'merchant_category', 'is_fraud', 'location']

📈 Sample t

In [7]:
# Fix timestamp issue and retrieve features with correct timestamps
print("🔧 FIXING TIMESTAMP ISSUE AND RETRIEVING FEATURES")
print("="*60)

# Use timestamps that match our actual data ranges
customer_timestamp = pd.Timestamp('2022-01-01')  # Within customer data range (2020-2022)
transaction_timestamp = pd.Timestamp('2025-08-01')  # Within transaction data range (2025)

print(f"📅 Using data-appropriate timestamps:")
print(f"   Customer data timestamp: {customer_timestamp}")
print(f"   Transaction data timestamp: {transaction_timestamp}")

# Create entity dataframes with correct timestamps
customer_entity_df = pd.DataFrame({
    'customer_id': sample_customers,
    'event_timestamp': [customer_timestamp] * len(sample_customers)
})

transaction_entity_df = pd.DataFrame({
    'customer_id': sample_customers,
    'event_timestamp': [transaction_timestamp] * len(sample_customers)
})

print(f"\n📋 Customer Entity DataFrame:")
print(customer_entity_df)
print(f"\n📋 Transaction Entity DataFrame:")
print(transaction_entity_df)


🔧 FIXING TIMESTAMP ISSUE AND RETRIEVING FEATURES
📅 Using data-appropriate timestamps:
   Customer data timestamp: 2022-01-01 00:00:00
   Transaction data timestamp: 2025-08-01 00:00:00

📋 Customer Entity DataFrame:
   customer_id event_timestamp
0  CUST_000001      2022-01-01
1  CUST_000002      2022-01-01
2  CUST_000003      2022-01-01
3  CUST_000004      2022-01-01
4  CUST_000005      2022-01-01

📋 Transaction Entity DataFrame:
   customer_id event_timestamp
0  CUST_000001      2025-08-01
1  CUST_000002      2025-08-01
2  CUST_000003      2025-08-01
3  CUST_000004      2025-08-01
4  CUST_000005      2025-08-01


## 5. Online Feature Retrieval (Real-time Inference)

Now let's retrieve features from the online store for real-time model inference. This is what you'd use in production for serving models.


In [8]:
# Retrieve customer features with correct timestamp
print("🔄 Retrieving customer features with correct timestamp...")

try:
    customer_df = fs.get_historical_features(
        entity_df=customer_entity_df,
        features=[
            "customer_behavioral_profile:age",
            "customer_behavioral_profile:income", 
            "customer_behavioral_profile:credit_score",
            "customer_behavioral_profile:account_tenure_days",
            "customer_behavioral_profile:risk_profile",
            "customer_behavioral_profile:customer_segment"
        ]
    ).to_df()
    
    print("✅ Successfully retrieved customer behavioral profile features!")
    print(f"📊 Customer DataFrame shape: {customer_df.shape}")
    print(f"📋 Columns: {list(customer_df.columns)}")
    if len(customer_df) > 0:
        print(f"\n📈 Sample customer data:")
        print(customer_df.head())
    else:
        print("⚠️  No customer data returned - checking TTL settings...")
        # Check TTL for customer_behavioral_profile
        fv = fs.get_feature_view("customer_behavioral_profile")
        print(f"   TTL: {fv.ttl}")
        print(f"   Current time: {datetime.now()}")
        print(f"   Data timestamp: {customer_timestamp}")
        print(f"   Time difference: {datetime.now() - customer_timestamp}")
    
except Exception as e:
    print(f"❌ Error retrieving customer features: {e}")

# Retrieve transaction features with correct timestamp
print(f"\n🔄 Retrieving transaction features with correct timestamp...")

try:
    transaction_df = fs.get_historical_features(
        entity_df=transaction_entity_df,
        features=[
            "transaction_7d_aggregations:amount",
            "transaction_7d_aggregations:transaction_type",
            "transaction_7d_aggregations:merchant_category",
            "transaction_7d_aggregations:is_fraud",
            "transaction_7d_aggregations:location"
        ]
    ).to_df()
    
    print("✅ Successfully retrieved transaction features!")
    print(f"📊 Transaction DataFrame shape: {transaction_df.shape}")
    print(f"📋 Columns: {list(transaction_df.columns)}")
    if len(transaction_df) > 0:
        print(f"\n📈 Sample transaction data:")
        print(transaction_df.head())
    else:
        print("⚠️  No transaction data returned - checking TTL settings...")
        # Check TTL for transaction_7d_aggregations
        fv = fs.get_feature_view("transaction_7d_aggregations")
        print(f"   TTL: {fv.ttl}")
        print(f"   Current time: {datetime.now()}")
        print(f"   Data timestamp: {transaction_timestamp}")
        print(f"   Time difference: {datetime.now() - transaction_timestamp}")
    
except Exception as e:
    print(f"❌ Error retrieving transaction features: {e}")

print("\n" + "="*60)
print("📊 FIXED OFFLINE FEATURE RETRIEVAL COMPLETE")
print("="*60)


🔄 Retrieving customer features with correct timestamp...
✅ Successfully retrieved customer behavioral profile features!
📊 Customer DataFrame shape: (0, 8)
📋 Columns: ['customer_id', 'event_timestamp', 'age', 'income', 'credit_score', 'account_tenure_days', 'risk_profile', 'customer_segment']
⚠️  No customer data returned - checking TTL settings...
   TTL: 180 days, 0:00:00
   Current time: 2025-09-16 21:42:07.294032
   Data timestamp: 2022-01-01 00:00:00
   Time difference: 1354 days 21:42:07.294057

🔄 Retrieving transaction features with correct timestamp...
✅ Successfully retrieved transaction features!
📊 Transaction DataFrame shape: (5, 7)
📋 Columns: ['customer_id', 'event_timestamp', 'amount', 'transaction_type', 'merchant_category', 'is_fraud', 'location']

📈 Sample transaction data:
   customer_id           event_timestamp   amount transaction_type  \
0  CUST_000005 2025-08-01 00:00:00+00:00   256.34       DEBIT_CARD   
1  CUST_000003 2025-08-01 00:00:00+00:00   203.96           

## 5. Online Feature Retrieval (Real-time Inference)

Now let's retrieve features from the online store for real-time model inference. The online store should have the most recent materialized features.


In [9]:
# Online feature retrieval for real-time inference
print("🔄 Retrieving features from online store for real-time inference...")

# Get features for a specific customer (simulating real-time inference)
# Use a customer with recent data in all datasets
customer_id = "CUST_000465"  # Customer with recent data in all datasets
print(f"🎯 Getting features for customer: {customer_id}")
print(f"   This customer has recent data in all datasets (within TTL windows)")

try:
    # Get customer behavioral profile features
    customer_features = fs.get_online_features(
        features=[
            "customer_behavioral_profile:age",
            "customer_behavioral_profile:income",
            "customer_behavioral_profile:credit_score",
            "customer_behavioral_profile:account_tenure_days",
            "customer_behavioral_profile:risk_profile",
            "customer_behavioral_profile:customer_segment"
        ],
        entity_rows=[{"customer_id": customer_id}]
    ).to_dict()
    
    print("✅ Successfully retrieved customer behavioral profile features from online store!")
    print(f"📊 Customer {customer_id} behavioral features:")
    for feature_name, value in customer_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error retrieving behavioral profile features: {e}")

# Also try getting basic demographic features
print(f"\n🔄 Getting basic demographic features...")
try:
    demo_features = fs.get_online_features(
        features=[
            "customer_demographics_fv:age",
            "customer_demographics_fv:income",
            "customer_demographics_fv:credit_score"
        ],
        entity_rows=[{"customer_id": customer_id}]
    ).to_dict()
    
    print("✅ Successfully retrieved basic demographic features!")
    print(f"📊 Customer {customer_id} demographic features:")
    for feature_name, value in demo_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error retrieving demographic features: {e}")

print("\n" + "="*60)
print("⚡ ONLINE FEATURE RETRIEVAL COMPLETE")
print("="*60)


🔄 Retrieving features from online store for real-time inference...
🎯 Getting features for customer: CUST_000465
   This customer has recent data in all datasets (within TTL windows)
✅ Successfully retrieved customer behavioral profile features from online store!
📊 Customer CUST_000465 behavioral features:
   credit_score: None
   account_tenure_days: None
   age: None
   customer_segment: None
   risk_profile: None
   income: None

🔄 Getting basic demographic features...
✅ Successfully retrieved basic demographic features!
📊 Customer CUST_000465 demographic features:
   credit_score: None
   age: None
   income: None

⚡ ONLINE FEATURE RETRIEVAL COMPLETE


In [10]:
# Online feature retrieval for real-time inference
print("🔄 Retrieving features from online store for real-time inference...")

# Get features for a specific customer (simulating real-time inference)
customer_id = "CUST_000465"
print(f"🎯 Getting features for customer: {customer_id}")

try:
    # Get customer behavioral profile features
    customer_features = fs.get_online_features(
        features=[
            "customer_behavioral_profile:age",
            "customer_behavioral_profile:income",
            "customer_behavioral_profile:credit_score",
            "customer_behavioral_profile:account_tenure_days",
            "customer_behavioral_profile:risk_profile",
            "customer_behavioral_profile:customer_segment"
        ],
        entity_rows=[{"customer_id": customer_id}]
    ).to_dict()
    
    print("✅ Successfully retrieved customer behavioral profile features from online store!")
    print(f"📊 Customer {customer_id} behavioral features:")
    for feature_name, value in customer_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error retrieving behavioral profile features: {e}")

# Also try getting basic demographic features
print(f"\n🔄 Getting basic demographic features...")
try:
    demo_features = fs.get_online_features(
        features=[
            "customer_demographics_fv:age",
            "customer_demographics_fv:income",
            "customer_demographics_fv:credit_score"
        ],
        entity_rows=[{"customer_id": customer_id}]
    ).to_dict()
    
    print("✅ Successfully retrieved basic demographic features!")
    print(f"📊 Customer {customer_id} demographic features:")
    for feature_name, value in demo_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error retrieving demographic features: {e}")

print("\n" + "="*60)
print("⚡ ONLINE FEATURE RETRIEVAL COMPLETE")
print("="*60)


🔄 Retrieving features from online store for real-time inference...
🎯 Getting features for customer: CUST_000465
✅ Successfully retrieved customer behavioral profile features from online store!
📊 Customer CUST_000465 behavioral features:
   credit_score: None
   account_tenure_days: None
   age: None
   customer_segment: None
   risk_profile: None
   income: None

🔄 Getting basic demographic features...
✅ Successfully retrieved basic demographic features!
📊 Customer CUST_000465 demographic features:
   credit_score: None
   age: None
   income: None

⚡ ONLINE FEATURE RETRIEVAL COMPLETE


## 6. Using Feature Services

Feature services allow us to group related features together for specific ML models. Let's demonstrate how to use them.


In [11]:
# Demonstrate feature services
print("🔧 FEATURE SERVICES DEMONSTRATION")
print("="*50)

# List available feature services
feature_services = fs.list_feature_services()
print(f"📋 Available Feature Services ({len(feature_services)}):")
for fs_service in feature_services:
    print(f"   - {fs_service.name}")

# Define a customer ID for feature service demonstration
# Use a customer that has recent data in all datasets (within TTL windows)
customer_id = "CUST_000465"  # Customer with recent data in all datasets
print(f"\n🎯 Using customer ID: {customer_id}")
print(f"   This customer has recent data in all datasets (within TTL windows)")

# Use the Customer Charter Feature Service
print(f"\n🎯 Using 'customer_charter_service' service...")
try:
    charter_features = fs.get_online_features(
        features=fs.get_feature_service("customer_charter_service"),
        entity_rows=[{"customer_id": customer_id}],
        full_feature_names=True
    ).to_dict()
    
    print("✅ Successfully retrieved features using feature service!")
    print(f"📊 Customer Charter features for customer {customer_id}:")
    for feature_name, value in charter_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error using customer_charter_service: {e}")

# Try another feature service - Customer Behavior Service
print(f"\n🎯 Using 'customer_behavior_service' service...")
try:
    behavior_features = fs.get_online_features(
        features=fs.get_feature_service("customer_behavior_service"),
        entity_rows=[{"customer_id": customer_id}],
        full_feature_names=True
    ).to_dict()
    
    print("✅ Successfully retrieved features using customer behavior service!")
    print(f"📊 Customer Behavior features for customer {customer_id}:")
    for feature_name, value in behavior_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
            
except Exception as e:
    print(f"❌ Error using customer_behavior_service: {e}")

print("\n" + "="*60)
print("🔧 FEATURE SERVICES DEMONSTRATION COMPLETE")
print("="*60)


🔧 FEATURE SERVICES DEMONSTRATION
📋 Available Feature Services (8):
   - customer_behavior_service
   - atm_optimization_service
   - call_prediction_service
   - customer_charter_service
   - branch_optimization_service
   - comprehensive_banking_service
   - transaction_prediction_service
   - risk_compliance_service

🎯 Using customer ID: CUST_000465
   This customer has recent data in all datasets (within TTL windows)

🎯 Using 'customer_charter_service' service...
✅ Successfully retrieved features using feature service!
📊 Customer Charter features for customer CUST_000465:
   customer_demographics_fv__credit_score: None
   customer_demographics_fv__age: None
   customer_demographics_fv__income: None
   customer_behavioral_profile__credit_score: None
   customer_behavioral_profile__account_tenure_days: None
   customer_behavioral_profile__age: None
   customer_behavioral_profile__customer_segment: None
   customer_behavioral_profile__risk_profile: None
   customer_behavioral_profile__

## 7. On-Demand Feature Transformations

On-demand features are computed in real-time during feature retrieval, allowing for dynamic transformations based on current data. This is particularly useful for banking scenarios where you need to calculate risk scores, fraud probabilities, or customer segments on-the-fly.

**Note**: On-demand features are experimental in Feast and can be challenging to implement. For this demo, we'll show the concept using Python transformations that could be implemented as on-demand features.


In [12]:
# On-Demand Feature Transformations Demo
print("🔄 ON-DEMAND FEATURE TRANSFORMATIONS DEMO")
print("="*60)

# Get some customer data for transformation
customer_data = fs.get_online_features(
    features=[
        "customer_demographics_fv:age",
        "customer_demographics_fv:income", 
        "customer_demographics_fv:credit_score"
    ],
    entity_rows=[{"customer_id": customer_id}]
).to_dict()

print(f"📊 Raw customer data for {customer_id}:")
for feature_name, value in customer_data.items():
    if feature_name != 'customer_id':
        print(f"   {feature_name}: {value[0]}")

# 1. Risk Scoring Transformation
print(f"\n🎯 1. RISK SCORING TRANSFORMATION")
print("-" * 40)

def calculate_risk_score(age, income, credit_score):
    """Calculate comprehensive risk score based on multiple factors"""
    # Normalize factors (0-1 scale)
    age_risk = min(age / 100, 1.0)  # Higher age = slightly higher risk
    income_stability = min(income / 200000, 1.0)  # Higher income = lower risk
    credit_quality = min(credit_score / 850, 1.0)  # Higher credit = lower risk
    
    # Weighted risk calculation (0-100 scale, where 100 is lowest risk)
    risk_score = (credit_quality * 0.5 + income_stability * 0.3 + (1.0 - age_risk) * 0.2) * 100
    return max(0, min(100, risk_score))

age = customer_data['customer_demographics_fv__age'][0]
income = customer_data['customer_demographics_fv__income'][0]
credit_score = customer_data['customer_demographics_fv__credit_score'][0]

risk_score = calculate_risk_score(age, income, credit_score)
print(f"   Age: {age}")
print(f"   Income: ${income:,.2f}")
print(f"   Credit Score: {credit_score}")
print(f"   🎯 Calculated Risk Score: {risk_score:.1f}/100")
print(f"   Risk Level: {'LOW' if risk_score > 70 else 'MEDIUM' if risk_score > 40 else 'HIGH'}")

# 2. Fraud Detection Transformation
print(f"\n🎯 2. FRAUD DETECTION TRANSFORMATION")
print("-" * 40)

# Get transaction data
transaction_data = fs.get_online_features(
    features=[
        "transaction_7d_aggregations:amount",
        "transaction_7d_aggregations:transaction_type",
        "transaction_7d_aggregations:location"
    ],
    entity_rows=[{"customer_id": customer_id}]
).to_dict()

def calculate_fraud_probability(amount, transaction_type, location):
    """Calculate fraud probability based on transaction patterns"""
    fraud_score = 0.0
    
    # Amount-based risk
    if amount > 1000:  # High amount transactions
        fraud_score += 30
    elif amount > 500:
        fraud_score += 15
    
    # Transaction type risk
    high_risk_types = ['ATM_WITHDRAWAL', 'CASH_ADVANCE', 'WIRE_TRANSFER']
    if transaction_type in high_risk_types:
        fraud_score += 25
    
    # Location risk
    if location == 'INTERNATIONAL':
        fraud_score += 35
    elif location == 'ONLINE':
        fraud_score += 10
    
    return min(100, fraud_score)

amount = transaction_data['transaction_7d_aggregations__amount'][0]
txn_type = transaction_data['transaction_7d_aggregations__transaction_type'][0]
location = transaction_data['transaction_7d_aggregations__location'][0]

fraud_prob = calculate_fraud_probability(amount, txn_type, location)
print(f"   Transaction Amount: ${amount:.2f}")
print(f"   Transaction Type: {txn_type}")
print(f"   Location: {location}")
print(f"   🎯 Fraud Probability: {fraud_prob:.1f}%")
print(f"   Risk Level: {'HIGH' if fraud_prob > 50 else 'MEDIUM' if fraud_prob > 20 else 'LOW'}")

# 3. Customer Segmentation Transformation
print(f"\n🎯 3. CUSTOMER SEGMENTATION TRANSFORMATION")
print("-" * 40)

def segment_customer(age, income, credit_score):
    """Segment customer based on demographics and behavior"""
    if income > 100000 and credit_score > 750:
        return "PREMIUM"
    elif income > 50000 and credit_score > 650:
        return "ACTIVE"
    elif age > 65:
        return "SENIOR"
    elif age < 30:
        return "YOUNG_PROFESSIONAL"
    else:
        return "STANDARD"

segment = segment_customer(age, income, credit_score)
print(f"   🎯 Customer Segment: {segment}")

# 4. Creditworthiness Assessment
print(f"\n🎯 4. CREDITWORTHINESS ASSESSMENT")
print("-" * 40)

def assess_creditworthiness(income, credit_score, age):
    """Assess creditworthiness for loan applications"""
    # Income component (40% weight)
    income_score = min(income / 100000, 1.0) * 40
    
    # Credit score component (40% weight)
    credit_score_normalized = min(credit_score / 850, 1.0) * 40
    
    # Age stability component (20% weight)
    age_stability = 1.0 - abs(age - 40) / 40  # Peak at age 40
    age_score = max(0, age_stability) * 20
    
    creditworthiness = income_score + credit_score_normalized + age_score
    return max(0, min(100, creditworthiness))

creditworthiness = assess_creditworthiness(income, credit_score, age)
print(f"   🎯 Creditworthiness Score: {creditworthiness:.1f}/100")
print(f"   Loan Eligibility: {'APPROVED' if creditworthiness > 60 else 'CONDITIONAL' if creditworthiness > 40 else 'DENIED'}")

print(f"\n" + "="*60)
print("📊 ON-DEMAND TRANSFORMATIONS COMPLETE")
print("="*60)
print("💡 These transformations could be implemented as Feast on-demand features")
print("   for real-time computation during feature retrieval!")


🔄 ON-DEMAND FEATURE TRANSFORMATIONS DEMO
📊 Raw customer data for CUST_000465:
   credit_score: None
   age: None
   income: None

🎯 1. RISK SCORING TRANSFORMATION
----------------------------------------


KeyError: 'customer_demographics_fv__age'